In [1]:
import pandas as pd
from gensim.models import Word2Vec
from scoring import Scoring

# Opciones de palabras prototípicas

Lista Asela

In [14]:
lista_asela_df = pd.read_csv('data/palabras_asela.csv')
lista_asela_df.drop(columns=['Positiva o Negativa'],inplace=True)
lista_asela_df.rename(columns={'Unnamed: 0':'word' , 'pos-neg':'prescore'},inplace=True)

# Creamos el diccionario de prescores:
W0_asela_dict = dict(zip(lista_asela_df['word'].values,lista_asela_df['prescore'].values))
list(W0_asela_dict.items())[:5]

[('bien', 1), ('nerviosa', -1), ('mal', -1), ('contenta', 1), ('difícil', -1)]

Lista Alba

In [15]:
lista_alba_df = pd.read_csv('data/clinical-prescores.csv',index_col=0)
lista_alba_df['c-prescore'] = 0.2*lista_alba_df['c-prescore']
lista_alba_df.rename(columns={'c-prescore':'prescore'},inplace=True)

# Creamos el diccionario de prescores:
W0_alba_dict = dict(zip(lista_alba_df['word'].values,lista_alba_df['prescore'].values))
list(W0_alba_dict.items())[:5]

[('lormetazepam', -1.0),
 ('mg', -1.0),
 ('mirtazapina', -1.0),
 ('fluoxetina', -1.0),
 ('alprazolam', -1.0)]

Automatic Keyword Extraction

In [2]:
from SentimentKW import KW

msjs_df = pd.read_csv("data/sms_full_dataframe.csv",index_col=0)
msjs_df['Normalized Label'] = (2/5)*msjs_df['ds'].values-1

ake = KW(df=msjs_df,text_col_name="Texto",label_col_name="Normalized Label")

In [3]:
kw_dict = ake.get_kw()
kw_dict

/home/wini/Repositorios/SMS-experimentos-PostCue/SentimentKW.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  labels_tr_df = labels_tr_df.append(temp_tr_df,ignore_index=True)
/home/wini/Repositorios/SMS-experimentos-PostCue/SentimentKW.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  labels_tr_df = labels_tr_df.append(temp_tr_df,ignore_index=True)
/home/wini/Repositorios/SMS-experimentos-PostCue/SentimentKW.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  labels_tr_df = labels_tr_df.append(temp_tr_df,ignore_index=True)
/home/wini/Repositorios/SMS-experimentos-PostCue/SentimentKW.py:63: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inst

{'sábados': -0.27092827791919477,
 'lleve': -0.2599997838025331,
 'verbal': -0.18963616764856733,
 'bruscos': -0.08646647167633881,
 'exno': -0.16435134529542506,
 'armonía': -0.13906652294228275,
 'parece': -0.24416666666666673,
 'relativo': -0.16099623522224768,
 'sobras': -0.06321205588285582,
 'vicioso': -0.23953098411114893,
 'adelanto': -0.10952419745669582,
 'tranxilium': -0.11402555179585643,
 'buenos': -0.19846153846153852,
 'saldrá': -0.1924101347778082,
 'decirse': -0.08646647167633881,
 'creemos': -0.06321205588285582,
 'levantado': -0.23147286821705432,
 'jugar': -0.16769127735515932,
 'bajadas': -0.19004258632642726,
 'quejas': -0.18963616764856733,
 'dispuesto': -0.21492099000170967,
 'racha': -0.22922936078824524,
 'cristales': -0.18963616764856733,
 'ambulatorio': -0.3057135946992956,
 'persiguiendo': -0.08849687823599815,
 'padecer': -0.10952419745669582,
 'animó': -0.10952419745669582,
 'sufra': -0.22481282635848077,
 'total': -0.1765217390811947,
 'cogido': -0.25428

In [4]:
import pickle

with open('data/palabras_auto.pickle', 'wb') as handle:
    pickle.dump(kw_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('data/palabras_auto.pickle', 'rb') as handle:
    kw_dict = pickle.load(handle)

# Leer Mensajes

In [15]:
msjs_df = pd.read_csv("data/sms_full_dataframe.csv",index_col=0)
msjs_df['Normalized Label'] = (2/5)*msjs_df['ds'].values-1
msjs_df.sample(3)

,Llave,Paciente,Sexo,cal_vida,Texto,ds03,ds45,dp0,dp1,ds0,ds1,ds2,ds3,ds4,ds5,ds,dp,Normalized Label
5000,18341,12272,1,24.8,quiero hacer cosas hago pretendo hacer sensaci...,0,1,1,0.0,0,0,0,0,0,1,5,0.0,1.0
1780,10813,3850,1,29.0,semana siento regular,0,1,0,1.0,0,0,0,0,1,0,4,1.0,0.6
3977,16433,8508,2,67.8,horas sueño sueño día dolor cabeza energía ok,0,1,1,0.0,0,0,0,0,0,1,5,0.0,1.0


# Leer embeddings

Cargamos el modelo de embeddings

In [6]:
w2v_model = Word2Vec.load("data/word2vec.model")

In [7]:
W0_dict = W0_asela_dict
# W0_dict = W0_alba_dict
# W0_dict = kw_dict  

puntaje = Scoring(w2vmodel=w2v_model,W0=W0_dict)

In [8]:
puntaje.build_neighbors(alpha=0.5)

In [9]:
scores_df = puntaje.transform(df=msjs_df,text_col="Texto",label_col="Normalized Label",beta1=0.5,beta2=5.5)
scores_df.sample(5)

,word,score
5532,sanitarias,-0.002336
3495,implica,-0.002540
10383,arropada,-0.001330
9805,misión,-0.042937
241,sentimental,-0.040082


Las más negativas

In [10]:
scores_df.sort_values(by='score').head(30)

,word,score
10777,arranques,-0.673374
10715,soportaba,-0.632409
10707,volumen,-0.632409
4347,lee,-0.627665
4360,muerda,-0.627665
10767,malditas,-0.627665
10734,aterrorizado,-0.627665
6082,joana,-0.627665
10743,refugiarme,-0.616366
2509,coño,-0.614529


Las más positivas

In [11]:
scores_df.sort_values(by='score',ascending=False).head(30)

,word,score
11891,empezaron,0.000000
9654,sentimentales,0.000000
9766,entrevistan,0.000000
8645,arduas,0.000000
3244,desconcertada,0.000000
7459,hartó,0.000000
7531,alegar,0.000000
6084,sufran,0.000000
9198,irán,0.000000
9367,estimulen,0.000000


# Representaciones de palabras

In [12]:
X_words_rep = puntaje.get_words_representations(mode='mean')
X_words_rep[:3,:5]

array([[0.01084175, 0.01784768, 0.02210057, 0.00433742, 0.0140518 ],
       [0.01010547, 0.01250294, 0.05826921, 0.00306403, 0.00014977],
       [0.01043482, 0.02030134, 0.03622553, 0.00920144, 0.00885838]])

In [13]:
X_msjs_rep = puntaje.get_texts_representations_Nmean(n=3)
X_msjs_rep[:3,:5]

array([[ 0.03045575, -0.01958298,  0.07817543,  0.01158943,  0.001033  ],
       [-0.00058585,  0.00146881, -0.00188204,  0.00263672, -0.00486017],
       [ 0.02298816,  0.00451415,  0.02315882, -0.00559644, -0.01133282]])

# Clasification

In [16]:
y = msjs_df['ds'].values
y

array([0, 1, 3, ..., 5, 4, 5])

In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_msjs_rep, y, random_state=331,train_size=0.75)

In [20]:
from sklearn.svm import SVC

clf = SVC(kernel='rbf')
clf.fit(X=X_train,y=y_train)

y_pred = clf.predict(X_test)

In [22]:
from sklearn.metrics import accuracy_score, recall_score, precision_score

print(f"Accuracy: {accuracy_score(y_test,y_pred)}")
print(f"Recall: {recall_score(y_test,y_pred,average='macro')}")
print(f"Precision: {precision_score(y_test,y_pred,average='macro')}")

Accuracy: 0.5338674435542607
Recall: 0.3061458510197527
Precision: 0.4145830908164874
